# Key Levels Detection

In [9]:
from Utils.YahooDataCollector import YahooDataCollector
stock = YahooDataCollector('AstraMicro.NS')

[*********************100%%**********************]  1 of 1 completed


In [8]:
stock.hist

,time,open,high,low,close,volume
Date,,,,,,
2004-11-01,26.733334,26.733334,26.719999,26.719999,22.610746,375
2004-11-02,26.719999,27.333332,25.933332,26.013332,22.012758,4770
2004-11-03,26.666666,26.666666,25.466665,25.493334,21.572725,111150
2004-11-04,25.600000,26.666666,25.600000,26.666666,22.565613,9075
2004-11-05,26.366667,28.000000,26.366667,27.973333,23.671328,28005
...,...,...,...,...,...,...
2024-04-19,699.700012,702.000000,677.150024,696.450012,696.450012,407682
2024-04-22,700.900024,714.750000,693.000000,709.200012,709.200012,256106
2024-04-23,709.650024,720.200012,705.099976,707.799988,707.799988,374270


In [10]:
import pandas as pd
df = stock.hist
df.columns=['time', 'open', 'high', 'low', 'close', 'volume']
#Check if NA values are in data
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.head(10)

,time,open,high,low,close,volume
0,26.733334,26.733334,26.719999,26.719999,22.610746,375
1,26.719999,27.333332,25.933332,26.013332,22.012758,4770
2,26.666666,26.666666,25.466665,25.493334,21.572725,111150
3,25.600000,26.666666,25.600000,26.666666,22.565613,9075
4,26.366667,28.000000,26.366667,27.973333,23.671331,28005
5,29.333332,29.333332,28.333332,28.613333,24.212906,21330
6,29.133333,29.133333,27.466665,27.706667,23.445673,5175
7,28.933332,28.933332,27.873333,27.879999,23.592352,1980
8,27.873333,28.586666,27.680000,27.873333,23.586706,17025
9,28.139999,28.139999,27.866667,27.866667,23.581064,2250


In [11]:
def pivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.low[l]>df1.low[i]):
            pividlow=0
        if(df1.high[l]<df1.high[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0
    
df['pivot'] = df.apply(lambda x: pivotid(df, x.name,10,10), axis=1)

C:\Users\TI-SJL-0008\AppData\Local\Temp\ipykernel_25792\2247444896.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Pivot Candles Vizualisation

In [12]:
df

,time,open,high,low,close,volume,pivot
0,26.733334,26.733334,26.719999,26.719999,22.610746,375,0
1,26.719999,27.333332,25.933332,26.013332,22.012758,4770,0
2,26.666666,26.666666,25.466665,25.493334,21.572725,111150,0
3,25.600000,26.666666,25.600000,26.666666,22.565613,9075,0
4,26.366667,28.000000,26.366667,27.973333,23.671331,28005,0
...,...,...,...,...,...,...,...
4796,699.700012,702.000000,677.150024,696.450012,696.450012,407682,0
4797,700.900024,714.750000,693.000000,709.200012,709.200012,256106,0
4798,709.650024,720.200012,705.099976,707.799988,707.799988,374270,0
4799,711.750000,718.000000,697.150024,712.599976,712.599976,346887,0


In [13]:
import numpy as np
def pointpos(x):
    if x['pivot']==1:
        return x['low']-1e-3
    elif x['pivot']==2:
        return x['high']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

C:\Users\TI-SJL-0008\AppData\Local\Temp\ipykernel_25792\3755922836.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
dfpl = df[-300:-1]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'],
                increasing_line_color= 'green', 
                decreasing_line_color= 'red')])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
# fig.update_layout(paper_bgcolor='black', plot_bgcolor='black')

fig.show()